In [7]:
import pycaret
import pandas as pd
from pycaret.classification import *
pycaret.__version__


'3.3.2'

In [8]:
def process_dataframe(df):
    # Extract columns that contain only numeric values
    numeric_cols = df.select_dtypes(include=[float, int]).columns.tolist()
    
    # Check if 'species_name' and 'set' columns exist in the DataFrame
    other_cols = ['species_name']
    for col in other_cols:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' does not exist in the DataFrame")
    
    # Merge numeric columns and other specified columns
    selected_cols = numeric_cols + other_cols
    
    # Create a new DataFrame
    new_df = df[selected_cols]
    
    return new_df

def split_dataset(csv_path):
    df = pd.read_csv(csv_path)
    train_val_set = df[df['set'].isin(['train', 'val'])]
    train_val_set = process_dataframe(train_val_set)  # The original data contains columns other than features and targets; this function removes them, retaining only features and targets.
    test_set = df[df['set'].isin(['test'])]
    test_set = process_dataframe(test_set)
    return train_val_set, test_set

In [9]:
import glob
import os
import time
import gc

csv_paths = glob.glob('./FusedFeatures/*.csv')

for csv_path in csv_paths:
    
    train_val_set, test_set = split_dataset(csv_path)
    
    exp = ClassificationExperiment()

    session_id = int(time.time())
    features_type = os.path.split(csv_path)[1].split('.')[0]
    
    exp.setup(train_val_set, target='species_name', session_id=session_id, normalize=True, pca=True,
              experiment_name=features_type, use_gpu=True)

    # models = ['gpc','gbc','lightgbm'] these models are very slow ,and performence not well in our preexperiment
    # model_names = ['ridge', 'rbfsvm', 'lda', 'lr', 'mlp', 'rf', 'knn', 'nb', 'dt', 'svm', 'qda', 'ada', 'et', 'dummy']
    model_names = ['catboost']
 
    trained_models = exp.compare_models(include=model_names, fold=5, n_select=1)

    save_folder = os.path.join('./models/gpu/', features_type)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    
    for trained_model in [trained_models]:
        model_name = trained_model.__class__.__name__

        # Predict using the trained model and save the prediction result
        predict_result_train_model = exp.predict_model(trained_model, test_set)
        predict_result_train_model.to_csv(os.path.join(save_folder, model_name + '_train.csv'))
        exp.save_model(trained_model, os.path.join(save_folder, model_name + '_train'))

        # Tune the model, predict using the tuned model, and save the prediction result
        tuned_model, tuner = exp.tune_model(trained_model, return_tuner=True, n_iter=20, fold=5, choose_better=True)
        predict_result_tune_model = exp.predict_model(tuned_model, test_set)
        predict_result_tune_model.to_csv(os.path.join(save_folder, model_name + '_tune.csv'))
        exp.save_model(tuned_model, os.path.join(save_folder, model_name + '_tune'))

        print(tuner)        

    exp.save_experiment(os.path.join(save_folder, 'experiment'))
    gc.collect()

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Description,Value
0,Session id,1741331404
1,Target,species_name
2,Target type,Multiclass
3,Target mapping,"BiJianGenGuanZao: 0, BianMianJiaoMaoZao: 1, BingHeNiXingGanZao: 2, BingJiJiaoMaoZao: 3, BoBiJiNeiYaZao: 4, BuShiShuangWeiZao: 5, ChaZhuangJiaoZao: 6, ChangTiaoFenBian: 7, ChangTiaoZaZhi: 8, CuCiJiaoZao: 9, DingDingChong: 10, DongWuCanZhi: 11, DuanJiaoWanJiaoZao: 12, DuoMaoLeiYouTi: 13, FuGanZao: 14, GangMaoGenGuanZao: 15, GeShiYuanShaiZao: 16, HaiYangYuanDuoJiaZao: 17, JiPiDongWuChangWanYouTi: 18, JianCiNiLingXingZao: 19, JiaoMaoZao: 20, JiaoMaoZao2: 21, JiaoMaoZao3: 22, JiaoZao1: 23, JuCaoPaLaZao: 24, KaShiJiaoMaoZao: 25, LingXingHaiXianZao: 26, LuoLeiYouTi: 27, ManZuLeiLiuZhiYouTi: 28, MiLianJiaoMaoZao: 29, NaoZuLeiWuJieYouTi2: 30, NiLingChong: 31, QuZhouZao: 32, RaoZuLei: 33, RaoZuLeiWuJieYouTi1: 34, RouRuoJiaoMaoZao: 35, SanJiaoJiaoZao: 36, ShuangQiaoBeiLeiYouTi: 37, SiShiBianJiaZao: 38, SiShiJiNeiYaZao: 39, SongSanFenBian: 40, SongSanHunHeTi: 41, SuoJiaoZao: 42, TaXingGuanGaiZao: 43, TaiWuShiNiuQiaoZao: 44, TaiXianChong: 45, TaiYangChong: 46, UnknownLarval: 47, WeiLiYuanShaiZao: 48, XianMaoChong: 49, XuanLianJiaoMaoZao: 50, YeGuangZao: 51, YinDuYiGenGuanZao: 52, YouMeiXuShiZao: 53, YuanShaiZao(CeMian): 54, YuanShaiZao(zhengmian): 55, ZaZhi1: 56, ZhaiXiJiaoMaoZao: 57, ZhengHeXingZao: 58, ZhongHuaChiZhuangZao: 59, ZhongLeiGuTiaoZao: 60, ZhouXingZao: 61, ZhuNangChong: 62, egg: 63, haishenyouchong: 64"
4,Original data shape,"(8689, 1281)"
5,Transformed data shape,"(8689, 1281)"
6,Transformed train set shape,"(6082, 1281)"
7,Transformed test set shape,"(2607, 1281)"
8,Numeric features,1280
9,Preprocess,True


...

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,01:17:19
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9293,0.9994,0.9293,0.9300,0.9253,0.9262,0.9263
1,0.9334,0.9996,0.9334,0.9259,0.9260,0.9305,0.9306
2,0.9424,0.9996,0.9424,0.9431,0.9383,0.9399,0.9400
3,0.9424,0.9996,0.9424,0.9432,0.9374,0.9399,0.9400
4,0.9490,0.9990,0.9490,0.9491,0.9463,0.9468,0.9469
Mean,0.9393,0.9994,0.9393,0.9383,0.9347,0.9367,0.9368
Std,0.0070,0.0003,0.0070,0.0088,0.0080,0.0074,0.0073


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.9446,0.9995,0.9446,0.9425,0.9404,0.9423,0.9423


Transformation Pipeline and Model Successfully Saved
RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=Pipeline(memory=Memory(location=None),
                                      steps=[('label_encoding',
                                              TransformerWrapperWithInverse(exclude=None,
                                                                            include=None,
                                                                            transformer=LabelEncoder())),
                                             ('numerical_imputer',
                                              TransformerWrapper(exclude=None,
                                                                 include=['0',
                                                                          '1',
                                                                          '2',
                         